# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [10]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
openai = OpenAI()

In [11]:
# Get gpt-4o-mini to answer, with streaming
def stream_gpt(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": "You are ahelpful assistant."},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
        
stream_gpt(question)

This line of code is using a generator expression along with the `yield from` statement in Python. Let's break it down:

1. **Set Comprehension**: The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It iterates over a collection called `books` and extracts the `author` attribute from each `book`. 

    - `book.get("author")` tries to retrieve the value associated with the key `"author"` from each `book` (which is assumed to be a dictionary). If the key doesn't exist (if there is no author), `get` returns `None` by default.
    
    - The `if book.get("author")` condition filters out any books that don't have an author (i.e., it only keeps books for which `book.get("author")` is truthy).

2. **Set Creation**: The outer curly braces `{}` indicate that this is a set comprehension. Sets automatically handle duplicates, so if multiple books have the same author, each author will only be included once in the final set.

3. **Yield from**: The `yield from` statement is used to yield all values from the iterable that's provided to it. In this case, the iterable is the set created by the comprehension. Using `yield from` allows for a clean way to yield all the elements of the set one by one.

In summary, the entire line of code effectively does the following:

- It creates a set of unique authors from a list of books, ignoring any books that do not have an author.
- It then yields each unique author one by one.

### Why Use This Code?
1. **Efficiency**: By using a set comprehension, it efficiently gathers unique authors without manually checking for duplicates.
2. **Generator**: Using `yield from` makes the code act like a generator, which can be useful for iterating over authors in a memory-efficient way (if there are many authors, they are yielded one at a time rather than stored all in memory).
3. **Readability**: The code is concise and leverages Python's expressive capabilities, making it easy to understand at a glance what it’s doing.

### Example
Consider if `books` is defined as follows:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},
    {"title": "Book 4"},  # No author
]


The code will yield:
- "Author A"
- "Author B"

(Note: No duplicates and ignoring the entry without an author.)

In [ ]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url='http://192.168.0.111:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=[question]
)

print(response.choices[0].message.content)